In [ ]:
import os 

os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

import plotly.express as px
import pandas as pd
import numpy as np
import h3

from mirrorverse.utils import read_data_w_cache

# Model Evaluation

## Helpful Stuff

In [ ]:
MODELS = {
    'A': ('3_1_1', 'ab17d4ce30981b9d7630da4d7adbf7fd7cb88a9bfee2b37ed60254e097e8ffdc'),
    'B': ('3_1_3', 'e875c3a83c56925e0537b30c6f64d3219ffcd41c2298490d69eec4c25899119c'),
    'C': ('3_1_4', '00cf23b296999368ea18b82e33b8687c51e8c35e876afd325e26317cb69ea45b'),
    'D': ('3_7_2', 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'),
}

## Overall Model Results

In [ ]:
dfs = []
for model_name, (model_id, run_id) in MODELS.items():
    sql = f'''
    with likelihoods as (
        select
            _individual,
            '{model_name}' as model,
            _train, 
            avg(ln(probability)) as nll
        from 
            chinook_depth_inference_{model_id}
        where 
            run_id = '{run_id}'
            and _selected
        group by 
            1, 2, 3
    )
    select
        model,
        _train,
        avg(nll) as nll
    from 
        likelihoods
    group by 
        1, 2
    '''
    dfs.append(read_data_w_cache(sql))


sql = f'''
with probabilities as (
    select
        _individual,
        _decision,
        _train, 
        1.0 / cast(count(*) as double) as probability
    from 
        chinook_depth_inference_{model_id}
    where 
        run_id = '{run_id}'
    group by 
        1, 2, 3
), likelihoods as (
    select
        _individual,
        'Null' as model,
        _train, 
        avg(ln(probability)) as nll
    from 
        probabilities
    group by 
        1, 2, 3
)
select
    model,
    _train,
    avg(nll) as nll
from 
    likelihoods
group by 
    1, 2
'''
dfs.append(read_data_w_cache(sql))

data = pd.concat(dfs).sort_values(['_train', 'model']).reset_index(drop=True)
print(data.shape)
data

## Depth Skew

In [ ]:
model_name, (model_id, run_id) = next(iter(MODELS.items()))

sql = f'''
select
    depth_bin,
    count(*) as count
from 
    chinook_depth_inference_{model_id}
where
    run_id = '{run_id}'
    and _selected
group by 
    1
'''
data = read_data_w_cache(sql)
data['proportion'] = data['count'] / data['count'].sum()
data.sort_values('depth_bin', ascending=True)

In [ ]:
sql = f'''
select
    depth_bin,
    sum(probability) as count
from 
    chinook_depth_inference_3_7_2
where
    run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
group by 
    1
'''
data = read_data_w_cache(sql)
data['proportion'] = data['count'] / data['count'].sum()
data.sort_values('depth_bin', ascending=True)

In [ ]:
data[data['depth_bin'] <= 100].sum()

## Seasonality

In [ ]:
color_discrete_map = {
    "25.0": "#1b9e77",  # Green
    "50.0": "#d95f02",  # Orange
    "75.0": "#7570b3",  # Purple
    "100.0": "#e7298a",  # Pink
    "150.0": "#66a61e",  # Olive Green
    "200.0": "#e6ab02",  # Yellow-Orange
    "250.0": "#a6761d",  # Brown
    "300.0": "#666666",  # Gray
    "400.0": "#1f78b4",  # Blue
    "500.0": "#a6cee3",  # Light Blue
}

In [ ]:
model_id = '3_7_2'
run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
sql = f'''
select
    extract(month from time) as month,
    depth_bin,
    count(*) as count
from 
    chinook_depth_inference_{model_id}
where
    run_id = '{run_id}'
    and _selected
group by 
    1, 2
'''
actuals = read_data_w_cache(sql)
actuals['monthly_count'] = actuals.groupby('month')['count'].transform('sum')
actuals['proportion'] = actuals['count'] / actuals['monthly_count']
actuals['depth_bin'] = actuals['depth_bin'].astype(str)
actuals['case'] = 'actual'
px.bar(
    actuals, x='month', y='proportion', color='depth_bin', 
    color_discrete_map=color_discrete_map,
    category_orders={'depth_bin': color_discrete_map.keys()},
    title="Actual Proportion per Depth Bin by Month (Val)"
)

In [ ]:
model_id = '3_7_2'
run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
sql = f'''
select
    extract(month from time) as month,
    depth_bin,
    sum(probability) as count
from 
    chinook_depth_inference_{model_id}
where
    run_id = '{run_id}'
group by 
    1, 2
'''
val = read_data_w_cache(sql)
val['monthly_count'] = val.groupby('month')['count'].transform('sum')
val['proportion'] = val['count'] / val['monthly_count']
val['depth_bin'] = val['depth_bin'].astype(str)
val['case'] = 'predicted'
px.bar(
    val, x='month', y='proportion', color='depth_bin', 
    color_discrete_map=color_discrete_map,
    category_orders={'depth_bin': color_discrete_map.keys()},
    title="Predicted Proportion per Depth Bin by Month (Val)"
)

In [ ]:
df = pd.concat([val, actuals])
MONTHS = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}
df["month"] = df["month"].apply(lambda m: MONTHS[m])
fig = px.bar(
    df, x='month', y='proportion', color='depth_bin', 
    color_discrete_map=color_discrete_map,
    category_orders={'depth_bin': color_discrete_map.keys(), "case": ["actual", "predicted"], "month": list(MONTHS.values())},
    title="Proportion per Depth Bin by Month",
    facet_row="case", height=500, width=800
)
fig.show()

In [ ]:
fig.write_image("fig2_seasonality.png", format="png", scale=2)

## Diel

In [ ]:
model_id = '3_7_2'
run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'

sql = f'''
select
    extract(month from time) as month,
    cos_sun,
    sin_sun,
    _train,
    case 
        when depth_bin = 25 then 'shallow'
        else 'deep' 
    end as _case,
    sum(case when _selected then 1.0 else 0.0 end) as num_selected,
    sum(probability) as expected_num_selected
from 
    chinook_depth_inference_{model_id}
where
    run_id = '{run_id}'
group by 
    1, 2, 3, 4, 5
'''
data = read_data_w_cache(sql)
data['radians'] = np.arctan2(data['sin_sun'], data['cos_sun'])
data['radians'] = round(data['radians'] * 5) / 5
data = data.groupby(['month', '_train', 'radians', '_case'])[['num_selected', 'expected_num_selected']].sum().reset_index()
data['total_num_selected'] = data.groupby(['month', '_train', 'radians'])['num_selected'].transform('sum')
data['proportion'] = data['num_selected'] / data['total_num_selected']
data['predicted_proportion'] = data['expected_num_selected'] / data['total_num_selected']

data = pd.concat([
    data[data['_train']].assign(case='train').drop(columns=['predicted_proportion']),
    data[~data['_train']].assign(case='validation').drop(columns=['predicted_proportion']),
    data[~data['_train']].assign(case='predicted').drop(columns=['proportion']).rename(columns={'predicted_proportion': 'proportion'})
])

data["month"] = data["month"].apply(lambda m: MONTHS[m])

fig = px.scatter(
    data[data['_case'] == 'deep'], x='radians', y='proportion', color='case',
    facet_col='month', facet_col_wrap=4,
    title="Proportion Deeper than 25m by Time of Day", category_orders={'month': list(MONTHS.values())},
    height=900, width=1000, color_discrete_map={
        'train': 'blue', 'validation': 'orange', 'predicted': 'purple'
    }
)
fig.show()

In [ ]:
fig.write_image("fig3_diel.png", format="png", scale=2)

In [ ]:
sql = f'''
select
    cos_sun,
    sin_sun,
    tag_key,
    case 
        when depth_bin = 25 then 'shallow'
        else 'deep' 
    end as _case,
    sum(case when _selected then 1.0 else 0.0 end) as num_selected
from 
    chinook_depth_inference_{model_id}
where
    run_id = '{run_id}'
    and extract(month from time) = 8
group by 
    1, 2, 3, 4
'''
data = read_data_w_cache(sql)
data['radians'] = np.arctan2(data['sin_sun'], data['cos_sun'])
data['day'] = data['radians'] > 0
data = data.groupby(['tag_key', 'day', '_case'])[['num_selected']].sum().reset_index()
data['total_selected'] = data.groupby(['tag_key', 'day'])['num_selected'].transform('sum')
data['proportion'] = data['num_selected'] / data['total_selected']
print(data.shape)
data.head()

In [ ]:
px.histogram(data[data['_case'] == 'shallow'], x='proportion', color='day', barmode='group')

In [ ]:
data['tag_key'].nunique()

In [ ]:
data[data['day'] & (data['proportion'] > .70) & (data['_case'] == 'shallow')].reset_index(drop=True)

In [ ]:
data[data['day'] & (data['proportion'] < .15) & (data['_case'] == 'shallow')].reset_index(drop=True)

In [ ]:
data[data['day'] & (data['proportion'] < .3) & (data['_case'] == 'shallow')].reset_index(drop=True)

In [ ]:
data[~data['day'] & (data['proportion'] < .50) & (data['_case'] == 'shallow')].reset_index(drop=True)

In [ ]:
data[~data['day'] & (data['proportion'] > .85) & (data['_case'] == 'shallow')].reset_index(drop=True)

## Salinity

In [ ]:
sql = '''
select
    salinity
from 
    chinook_depth_inference_3_7_2
where 
    run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    and depth_bin = 25
'''
boundary = read_data_w_cache(sql)['salinity'].quantile(0.25)
print(boundary)

In [ ]:
sql = f'''
with surface_salinity as (
    select
        _individual,
        _decision,
        _train,
        salinity as surface_salinity
    from 
        chinook_depth_inference_3_7_2
    where 
        run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
        and depth_bin = 25
), max_depth_bins as (
    select
        _individual,
        _decision,
        _train,
        max(depth_bin) as max_depth_bin
    from 
        chinook_depth_inference_3_7_2
    where 
        run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    group by 
        1, 2, 3
)
select
    extract(month from time) as month,
    max_depth_bin,
    case 
        when surface_salinity < {boundary} then 'low'
        else 'high'
    end as surface_salinity,
    sum(case when _selected then 1.0 else 0.0 end) as num_selected,
    sum(probability) as expected_num_selected,
    count(*) as samples
from 
    chinook_depth_inference_3_7_2
    inner join surface_salinity
        using (_individual, _decision, _train)
    inner join max_depth_bins
        using (_individual, _decision, _train)
where 
    run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    and depth_bin = 25
group by 
    1, 2, 3      

'''
data = read_data_w_cache(sql)
data['proportion'] = data['num_selected'] / data['samples']
data['predicted_proportion'] = data['expected_num_selected'] / data['samples']
data = data.groupby(['month', 'surface_salinity'])['predicted_proportion'].mean().reset_index()
print(data.shape)
data.head()

In [ ]:
data["month"] = data["month"].apply(lambda m: MONTHS[m])
fig = px.bar(
    data, x='month', y='predicted_proportion', barmode='group', 
    color='surface_salinity', category_orders={"month": list(MONTHS.values())},
    title="Predicted Proportion Near Surface by Surface Salinity",
    height=400, width=800
)
fig.show()

In [ ]:
fig.write_image("fig4_salinity1.png", format="png", scale=2)

In [ ]:
sql = '''
select 
    _train,
    extract(month from time) as month,
    avg(ln(probability)) as loss
from
    chinook_depth_inference_3_7_2
where 
    run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    and _selected
group by 1, 2
order by 1
'''
salinity = read_data_w_cache(sql)
print(salinity.shape)
salinity.head()

In [ ]:
sql = '''
select 
    _train,
    extract(month from time) as month,
    avg(ln(probability)) as loss
from
    chinook_depth_inference_3_1_4
where 
    run_id = '00cf23b296999368ea18b82e33b8687c51e8c35e876afd325e26317cb69ea45b'
    and _selected
group by 1, 2
order by 1
'''
base = read_data_w_cache(sql)
base.head()

In [ ]:
df = base.merge(salinity, on=['month', '_train'], suffixes=('', '_w_salinity'))
df['difference'] = -(df['loss_w_salinity'] - df['loss'])
df.sort_values(['month', '_train']).reset_index(drop=True)

In [ ]:
df['month'] = df['month'].apply(lambda m: MONTHS[m])
fig = px.bar(
    df, x='month', y='difference', color='_train', barmode='group',
    category_orders={'month': list(MONTHS.values())},
    title='Variation in Change in Loss with Salinity as a Feature',
    height=400, width=800
)
fig.show()

In [ ]:
fig.write_image("fig5_salinity2.png", format="png", scale=2)

# Assessing Likelihood of Occupancy Near the Seafloor

## Spatial Distribution of Minimum Likelihood

In [ ]:
sql = '''
with risk as (
    select 
        time, 
        epoch,
        h3_index,
        depth_bin,
        max(elevation) as elevation,
        sum(probability) as risk
    from 
        chinook_depth_full_inference_3_7_2
    where 
        run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    group by 
        1, 2, 3, 4
), max_depth_bin as (
    select 
        time, 
        epoch,
        h3_index,
        max(depth_bin) as depth_bin
    from 
        chinook_depth_full_inference_3_7_2
    where 
        run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    group by 
        1, 2, 3
)
select 
    month(time) as month,
    h3_index,
    depth_bin,
    elevation,
    approx_percentile(risk, 0.05) as min_risk_month,
    approx_percentile(risk, 0.95) as max_risk_month
from 
    risk inner join max_depth_bin using (time, epoch, h3_index, depth_bin)
group by 
    1, 2, 3, 4
'''
data = read_data_w_cache(sql)
data['lat'] = data['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
data['lon'] = data['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
print(data.shape)
data.head()

In [ ]:
data = data[data['depth_bin'] + 100 > -data['elevation']]
print(data.shape)

In [ ]:
from shapely.geometry import Polygon, Point

poly = Polygon(
    [
        (-166, 54.4),
        (-160, 56),
        (-158, 57.2),
        (-153, 62),
        (-149, 62),
        (-146, 62),
        (-140, 60),
        (-136, 58.4),
        (-133, 57.5),
        (-132, 56.0),
        (-131, 55),
        (-125, 50.3),
        (-170, 52.5),
        (-166, 54.4),
    ]
)
data['inside_polygon'] = data.apply(lambda row: poly.contains(Point(row['lon'], row['lat'])), axis=1)
fig = px.scatter_mapbox(
    data[(data['month'] == 2)  & (data['depth_bin'] != 25.0)],
    lat='lat',
    lon='lon',
    color='inside_polygon',  # Color points by probability
    size_max=10,  # Adjust as needed
    zoom=3,  # Adjust zoom level
    mapbox_style="carto-positron",  # Choose a map style,
    range_color=[0.0, 0.3]
)
fig.show()

In [ ]:
data = data[data['inside_polygon']]
data = data[data['lon'] < -145]
data['size'] = 0.01

In [ ]:
fig = px.scatter_mapbox(
    data[(data['month'] == 2)  & (data['depth_bin'] != 25.0)],
    lat='lat',
    lon='lon',
    color='min_risk_month',  # Color points by probability
    size='size',  # Adjust as needed
    size_max=11,  # Adjust as needed
    zoom=4,  # Adjust zoom level
    mapbox_style="carto-positron",  # Choose a map style,
    title='Spatial Pattern of Minimal Probability in Bottom Depth Bin in February',
    #range_color=[0.0, 0.3],
    height=600,
    width=1000
)
fig.show()

In [ ]:
fig = px.scatter_mapbox(
    data[(data['month'] == 8) & (data['depth_bin'] != 25.0)],
    lat='lat',
    lon='lon',
    color='min_risk_month',  # Color points by probability
    size='size',  # Adjust as needed
    size_max=11,  # Adjust as needed
    zoom=4,  # Adjust zoom level
    mapbox_style="carto-positron",  # Choose a map style,
    title='Spatial Pattern of Minimal Probability in Bottom Depth Bin in August',
    #range_color=[0.0, 0.3],
    height=600,
    width=1000
)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create subplot figure with 1 row and 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('February', 'August'),
    specs=[[{'type': 'mapbox'}, {'type': 'mapbox'}]],
    horizontal_spacing=0.01
)

# Prepare February data
feb_data = data[(data['month'] == 2) & (data['depth_bin'] != 25.0)]

# Prepare August data
aug_data = data[(data['month'] == 8) & (data['depth_bin'] != 25.0)]

color_min = min(feb_data['min_risk_month'].min(), aug_data['min_risk_month'].min())
color_max = max(feb_data['min_risk_month'].max(), aug_data['min_risk_month'].max())


# Add February trace
fig.add_trace(
    go.Scattermapbox(
        lat=feb_data['lat'],
        lon=feb_data['lon'],
        mode='markers',
        marker=dict(
            size=feb_data['size'],
            sizemode='diameter',
            sizeref=2.*max(feb_data['size'])/30,
            color=feb_data['min_risk_month'],
            colorscale='plasma',  # Adjust colorscale as needed
            showscale=False,
            cmin=color_min,
            cmax=color_max,
        ),
        text=feb_data['min_risk_month'],
        hovertemplate='<b>Lat:</b> %{lat}<br><b>Lon:</b> %{lon}<br><b>Value:</b> %{text}<extra></extra>'
    ),
    row=1, col=1
)

# Add August trace
fig.add_trace(
    go.Scattermapbox(
        lat=aug_data['lat'],
        lon=aug_data['lon'],
        mode='markers',
        marker=dict(
            size=aug_data['size'],
            sizemode='diameter',
            sizeref=2.*max(aug_data['size'])/30,
            color=aug_data['min_risk_month'],
            colorscale='plasma',
            showscale=True,
            cmin=color_min,
            cmax=color_max,
            colorbar=dict(x=1.02, len=0.5)
        ),
        text=aug_data['min_risk_month'],
        hovertemplate='<b>Lat:</b> %{lat}<br><b>Lon:</b> %{lon}<br><b>Value:</b> %{text}<extra></extra>'
    ),
    row=1, col=2
)

# Update mapbox layouts
fig.update_mapboxes(
    style="carto-positron",
    zoom=4,
    center=dict(lat=feb_data['lat'].mean(), lon=feb_data['lon'].mean())
)

# Update overall layout
fig.update_layout(
    title_text='Spatial Distribution of Minimal Likelihood in Bottom Depth Bin',
    title_x=0.5,
    height=600,
    width=1600,
    showlegend=False
)

fig.show()

In [ ]:
fig.write_image("fig6_spatial_risk.png", format="png", scale=2)

## Full Year of Likelihoods

In [ ]:
h3_indices = {
    'Chignik': {
        'Coastal': '840ccebffffffff',
    }
}

sql = '''
select 
    epoch, h3_index, depth_bin, probability
from 
    chinook_depth_full_inference_3_7_2
where 
    run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    and h3_index = '{h3_index}'
'''
dfs = []
for place, cases in h3_indices.items():
    for case, h3_index in cases.items():
        data = read_data_w_cache(sql.format(h3_index=h3_index))
        data['time'] = pd.to_datetime(data['epoch'], unit='s', utc=False)
        # convert to alaska time
        data['time'] = data['time'].dt.tz_localize('UTC').dt.tz_convert('America/Anchorage')
        data['place'] = place
        data['case'] = case
        print(data.shape)
        dfs.append(data)
data = pd.concat(dfs).reset_index(drop=True)
print(data.shape)
data.head()

In [ ]:
color_discrete_map = {
    25.0: "#c7e9c0",  # Light Green
    50.0: "#a1d99b",  
    75.0: "#74c476",  
    100.0: "#41ab5d",  
    150.0: "#238b45",  
    200.0: "#1b7837",  
    250.0: "#0868ac",  
    300.0: "#08519c",  
    400.0: "#08306b",  # Deep Blue
    500.0: "#041c40",  # Darkest Blue (Deepest)
}

depth_order = sorted(color_discrete_map.keys())
data['depth_bin'] = pd.Categorical(data['depth_bin'], categories=depth_order, ordered=True)
data['likelihood'] = data['probability']

# Now plot with the correct legend order
fig = px.line(
    data.sort_values('time'), x='time', y='likelihood', color='depth_bin',
    color_discrete_map=color_discrete_map,
    category_orders={"depth_bin": sorted(color_discrete_map.keys())},
    title='Likelihood per Depth Bin near Chignik',
    height=400, width=1000
)

fig.show()


In [ ]:
fig.write_image("fig8_full_year.png", format="png", scale=2)

## Time of Day Minimization

In [ ]:
sql = '''
with risk as (
    select 
        month(time) as month, 
        epoch,
        h3_index,
        depth_bin,
        sin_sun,
        cos_sun,
        max(elevation) as elevation,
        sum(probability) as risk
    from 
        chinook_depth_full_inference_3_7_2
    where 
        run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
        and day(time) = 15
    group by 
        1, 2, 3, 4, 5, 6
), max_depth_bin as (
    select 
        month(time) as month, 
        epoch,
        h3_index,
        max(depth_bin) as depth_bin
    from 
        chinook_depth_full_inference_3_7_2
    where 
        run_id = 'fb3f06dc5fd0971a4e7cfdd2e5da5cca391a633f92528e79aa526df347ca0920'
    group by 
        1, 2, 3
), boundaries as (
    select 
        month,
        h3_index,
        depth_bin,
        elevation,
        approx_percentile(risk, 0.05) as min_risk_month,
        approx_percentile(risk, 0.95) as max_risk_month
    from 
        risk inner join max_depth_bin using (month, epoch, h3_index, depth_bin)
    group by 
        1, 2, 3, 4
), joined as (
    select 
        r.month,
        r.h3_index,
        r.sin_sun, 
        r.cos_sun,
        r.risk,
        b.depth_bin,
        b.elevation,
        b.min_risk_month,
        b.max_risk_month
    from 
        risk r
        inner join boundaries b 
            on r.month = b.month
            and r.h3_index = b.h3_index
)
select 
    month,
    h3_index,
    depth_bin,
    elevation,
    avg(sin_sun) as avg_sin_sun,
    avg(cos_sun) as avg_cos_sun
from 
    joined 
where 
    risk <= min_risk_month
group by 
    1, 2, 3, 4
'''
data = read_data_w_cache(sql)
data['lat'] = data['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
data['lon'] = data['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
print(data.shape)
data.head()

In [ ]:
data = data[data['depth_bin'] + 100 > -data['elevation']]
data['inside_polygon'] = data.apply(lambda row: poly.contains(Point(row['lon'], row['lat'])), axis=1)
data = data[data['inside_polygon']]
data = data[data['lon'] < -145]
data['size'] = 0.01

In [ ]:
data['sun_is_up'] = np.sign(data['avg_sin_sun'])
fig = px.scatter_mapbox(
    data[(data['month'] == 2) & (data['depth_bin'] != 25.0)],
    lat='lat',
    lon='lon',
    color='avg_sin_sun',  # Color points by probability
    size='size',  # Adjust as needed
    size_max=11,  # Adjust as needed
    zoom=4,  # Adjust zoom level
    mapbox_style="carto-positron",  # Choose a map style,
    title='Spatial Pattern of Time of Minimal Risk in February',
    range_color=[-1, 1],
    height=600,
    width=1000,
    color_continuous_scale='RdBu_r',  # Red to Blue color scale
)
fig.show()

In [ ]:
data['sun_is_up'] = np.sign(data['avg_sin_sun'])
fig = px.scatter_mapbox(
    data[(data['month'] == 8) & (data['depth_bin'] != 25.0)],
    lat='lat',
    lon='lon',
    color='avg_sin_sun',  # Color points by probability
    size='size',  # Adjust as needed
    size_max=11,  # Adjust as needed
    zoom=4,  # Adjust zoom level
    mapbox_style="carto-positron",  # Choose a map style,
    title='Spatial Pattern of Time of Minimal Risk in August',
    range_color=[-1, 1],
    height=600,
    width=1000,
    color_continuous_scale='RdBu_r',  # Red to Blue col.3or scale
)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create subplot figure with 1 row and 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('February', 'August'),
    specs=[[{'type': 'mapbox'}, {'type': 'mapbox'}]],
    horizontal_spacing=0.01
)

# Prepare February data
feb_data = data[(data['month'] == 2) & (data['depth_bin'] != 25.0)]

# Prepare August data
aug_data = data[(data['month'] == 8) & (data['depth_bin'] != 25.0)]

# Use fixed color range
color_min = -1
color_max = 1

# Add February trace
fig.add_trace(
    go.Scattermapbox(
        lat=feb_data['lat'],
        lon=feb_data['lon'],
        mode='markers',
        marker=dict(
            size=feb_data['size'],
            sizemode='diameter',
            sizeref=2.*max(feb_data['size'])/30,
            color=feb_data['avg_sin_sun'],
            colorscale='RdBu_r',
            showscale=False,
            cmin=color_min,
            cmax=color_max,
        ),
        text=feb_data['avg_sin_sun'],
        hovertemplate='<b>Lat:</b> %{lat}<br><b>Lon:</b> %{lon}<br><b>Value:</b> %{text}<extra></extra>'
    ),
    row=1, col=1
)

# Add August trace
fig.add_trace(
    go.Scattermapbox(
        lat=aug_data['lat'],
        lon=aug_data['lon'],
        mode='markers',
        marker=dict(
            size=aug_data['size'],
            sizemode='diameter',
            sizeref=2.*max(aug_data['size'])/30,
            color=aug_data['avg_sin_sun'],
            colorscale='RdBu_r',
            showscale=True,
            cmin=color_min,
            cmax=color_max,
            colorbar=dict(x=1.02, len=0.5)
        ),
        text=aug_data['avg_sin_sun'],
        hovertemplate='<b>Lat:</b> %{lat}<br><b>Lon:</b> %{lon}<br><b>Value:</b> %{text}<extra></extra>'
    ),
    row=1, col=2
)

# Update mapbox layouts
fig.update_mapboxes(
    style="carto-positron",
    zoom=4,
    center=dict(lat=feb_data['lat'].mean(), lon=feb_data['lon'].mean())
)

# Update overall layout
fig.update_layout(
    title_text='Spatial Pattern of Time of Minimal Likelihood',
    title_x=0.5,
    height=600,
    width=1600,
    showlegend=False
)

fig.show()

In [ ]:
fig.write_image("fig7_time_risk.png", format="png", scale=2)

## Paths

In [ ]:
sql = '''
select
    tag_key, epoch, longitude, latitude
from 
    mgietzmann_tag_tracks
where
    upload_key = 'mgietzmann'
'''
data = read_data_w_cache(sql)
data["month"] = pd.to_datetime(data["epoch"], unit="s").dt.month
print(data.shape)
data.head()

In [ ]:
fig = px.scatter_map(
    data, lat='latitude', lon='longitude',
    zoom=3, center=dict(lat=data['latitude'].mean(), lon=data['longitude'].mean()),
    height=700, width=1000, title="Chinook Salmon Tag Tracks", color="month"
)
fig.show()

In [ ]:
fig.write_image("fig1_tracks.png", format="png", scale=2)